In [ ]:
import json
from pathlib import Path

def fix_predictions_order_and_clips(pred_file, gt_file, output_file):
    # Load the ground truth and predictions
    with open(gt_file, "r") as fp:
        gt_annotations = json.load(fp)
    with open(pred_file, "r") as fp:
        model_predictions = json.load(fp)

    # Extract the video_uid list from the ground truth
    gt_video_uids = [v["video_uid"] for v in gt_annotations["videos"]]

    # Extract the clips for each video from the ground truth
    gt_clips_dict = {
        v["video_uid"]: {clip["clip_uid"]: clip for clip in v["clips"]}
        for v in gt_annotations["videos"]
    }

    # Extract video predictions and create a dictionary for faster lookup
    video_predictions = model_predictions["results"]["videos"]
    video_predictions_dict = {v["video_uid"]: v for v in video_predictions}

    # Combine predictions and extra data (empty entries for missing video_uids)
    combined_predictions = []

    for uid in gt_video_uids:
        if uid in video_predictions_dict:
            # Get the prediction for this video
            vpred = video_predictions_dict[uid]
            
            # Sort clips based on ground truth clip order
            if "clips" in vpred and uid in gt_clips_dict:
                gt_clips = gt_clips_dict[uid]
                vpred_clips_dict = {clip["clip_uid"]: clip for clip in vpred["clips"]}
                
                # Ensure all ground truth clips are present in predictions
                sorted_clips = []
                for clip_uid, gt_clip in gt_clips.items():
                    if clip_uid in vpred_clips_dict:
                        pred_clip = vpred_clips_dict[clip_uid]

                        # Ensure number of predictions matches the number of annotations
                        num_annotations = len(gt_clip["annotations"])
                        num_predictions = len(pred_clip["predictions"])

                        if num_predictions < num_annotations:
                            # Add empty predictions if there are fewer predictions than annotations
                            for _ in range(num_predictions, num_annotations):
                                pred_clip["predictions"].append({"query_sets": {}})
                        
                        sorted_clips.append(pred_clip)
                    else:
                        # Add an empty clip with the same number of empty predictions as annotations
                        sorted_clips.append({
                            "clip_uid": clip_uid,
                            "predictions": [{"query_sets": {}} for _ in range(len(gt_clip["annotations"]))]
                        })
                vpred["clips"] = sorted_clips
            else:
                # If no clips exist in predictions, create empty clips for the video
                vpred["clips"] = [
                    {
                        "clip_uid": clip_uid,
                        "predictions": [{"query_sets": {}} for _ in range(len(gt_clips_dict[uid][clip_uid]["annotations"]))]
                    }
                    for clip_uid in gt_clips_dict[uid]
                ]

            combined_predictions.append(vpred)
        else:
            # Add missing video entries as empty predictions
            combined_predictions.append({
                "video_uid": uid, 
                "split": "test", 
                "clips": [
                    {
                        "clip_uid": clip_uid,
                        "predictions": [{"query_sets": {}} for _ in range(len(gt_clips_dict[uid][clip_uid]["annotations"]))]
                    }
                    for clip_uid in gt_clips_dict[uid]
                ]
            })

    # Update the predictions in the model_predictions object
    model_predictions["results"]["videos"] = combined_predictions

    # Write the updated predictions to a new file
    with open(output_file, 'w') as f:
        json.dump(model_predictions, f)

    print(f"Updated predictions saved to: {output_file}")

# Paths to your input and output files
pred_file = '../outputs/batch/2024-10-13/52661/test_predictions.json'
gt_file = '/data/dataset/ego4d_temp/ego4d_data/v2/annotations/vq_test_unannotated.json'
output_file = Path(pred_file).with_name('test_predictions_sorted.json')

# Call the function to fix the order and clips
fix_predictions_order_and_clips(pred_file, gt_file, output_file)
